In [34]:
#downloading libraries
import pandas as pd
import numpy as np
import seaborn as sns
import sys, os
from matplotlib import pyplot as plt
from sklearn.utils import resample
from sklearn.metrics import accuracy_score 
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

In [35]:
#linking database
#voice df outlook
adult_df = pd.read_csv ('adult.csv')
adult_df.head()

,age,workclass,fnlwgt,education,education number,marital status,occupation,relationship,race,sex,capital_gain,capital loss,hours_per_week,native_country,label
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [36]:
adult_df.shape

(32561, 15)

In [37]:
adult_df[adult_df == '?'] = np.nan
adult_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   age               32561 non-null  int64 
 1   workclass         32561 non-null  object
 2   fnlwgt            32561 non-null  int64 
 3   education         32561 non-null  object
 4   education number  32561 non-null  int64 
 5   marital status    32561 non-null  object
 6   occupation        32561 non-null  object
 7   relationship      32561 non-null  object
 8   race              32561 non-null  object
 9   sex               32561 non-null  object
 10  capital_gain      32561 non-null  int64 
 11  capital loss      32561 non-null  int64 
 12  hours_per_week    32561 non-null  int64 
 13  native_country    32561 non-null  object
 14  label             32561 non-null  object
dtypes: int64(6), object(9)
memory usage: 3.7+ MB


In [38]:
#i tried multiple options after searching online, but above table is wrong. there are multiple '?' values and they are not being read
adult_df['native_country'] = adult_df['native_country'].replace('?',np.nan)
adult_df['workclass'] = adult_df['workclass'].replace('?',np.nan)
adult_df['occupation'] = adult_df['occupation'].replace('?',np.nan)

In [39]:
#converting all the data to numbers rather than objects
from sklearn.preprocessing import LabelEncoder
categorical = ['workclass','education', 'marital status', 'occupation', 'relationship',
               'race', 'sex','native_country','label']
label_encoder = LabelEncoder()
for col in categorical:
    label_encoder.fit(adult_df[col])
    adult_df[col] = label_encoder.transform(adult_df[col])


In [40]:
adult_df.head()


,age,workclass,fnlwgt,education,education number,marital status,occupation,relationship,race,sex,capital_gain,capital loss,hours_per_week,native_country,label
0,39,7,77516,9,13,4,1,1,4,1,2174,0,40,39,0
1,50,6,83311,9,13,2,4,0,4,1,0,0,13,39,0
2,38,4,215646,11,9,0,6,1,4,1,0,0,40,39,0
3,53,4,234721,1,7,2,6,0,2,1,0,0,40,39,0
4,28,4,338409,9,13,2,10,5,2,0,0,0,40,5,0


In [41]:
# we are going to drop fnlwgt, since it is weight bias of how many such people are there and for us it is a noisy data
# we will drop ,educational number does not affect the salary hence we drop it
# we do not take capital income and loss as majority of the entries are zero

In [42]:
adult_df.corr()

,age,workclass,fnlwgt,education,education number,marital status,occupation,relationship,race,sex,capital_gain,capital loss,hours_per_week,native_country,label
age,1.000000,0.003787,-0.076646,-0.010508,0.036527,-0.266288,-0.020947,-0.263698,0.028718,0.088832,0.077674,0.057775,0.068756,-0.001151,0.234037
workclass,0.003787,1.000000,-0.016656,0.023513,0.052085,-0.064731,0.254892,-0.090461,0.049742,0.095981,0.033835,0.012216,0.138962,-0.007690,0.051604
fnlwgt,-0.076646,-0.016656,1.000000,-0.028145,-0.043195,0.028153,0.001597,0.008931,-0.021291,0.026858,0.000432,-0.010252,-0.018768,-0.051966,-0.009463
education,-0.010508,0.023513,-0.028145,1.000000,0.359153,-0.038407,-0.021260,-0.010876,0.014131,-0.027356,0.030046,0.016746,0.055510,0.064288,0.079317
education number,0.036527,0.052085,-0.043195,0.359153,1.000000,-0.069304,0.109697,-0.094153,0.031838,0.012280,0.122630,0.079923,0.148123,0.050840,0.335154
marital status,-0.266288,-0.064731,0.028153,-0.038407,-0.069304,1.000000,-0.009654,0.185451,-0.068013,-0.129314,-0.043393,-0.034187,-0.190519,-0.023819,-0.199307
occupation,-0.020947,0.254892,0.001597,-0.021260,0.109697,-0.009654,1.000000,-0.075607,0.006763,0.080296,0.025505,0.017987,0.080383,-0.012543,0.075468
relationship,-0.263698,-0.090461,0.008931,-0.010876,-0.094153,0.185451,-0.075607,1.000000,-0.116055,-0.582454,-0.057919,-0.061062,-0.248974,-0.005507,-0.250918
race,0.028718,0.049742,-0.021291,0.014131,0.031838,-0.068013,0.006763,-0.116055,1.000000,0.087204,0.011145,0.018899,0.041910,0.137852,0.071846
sex,0.088832,0.095981,0.026858,-0.027356,0.012280,-0.129314,0.080296,-0.582454,0.087204,1.000000,0.048480,0.045567,0.229309,-0.008119,0.215980


In [43]:

selected_features = ['age','workclass','education','occupation','relationship','race','sex','hours_per_week','native_country']
x = adult_df[selected_features]
y = adult_df.label
print(x.shape)
print(y.shape)
xtrain, xtest, ytrain, ytest = train_test_split(x,y,test_size=0.20)

(32561, 9)
(32561,)


In [86]:
def test_metrics(ytest,ypred):
    cm = confusion_matrix(ytest,ypred)
    print('accuracy score', accuracy_score(ytest,ypred))
    print('confusion matrix\n', cm )
    print('CLS report\n', classification_report(ytest,ypred))
def validation(ytest,ypred):
    P,N = confusion_matrix(ytest,ypred)
    TP,FN = P
    FP,TN = N
    precision0 = TN/(TN+FN)
    precision1 = TP/(TP+FP)
    Recall0 = TN/(TN+FN)
    Recall1 = TP/(TP+FP)
    print("Validation\t precision\t \t Recall \t\t F1-score")
    print("1\t \t",precision0,"\t",Recall0,"\t",(2*precision0*Recall0/(precision0+Recall0)))
    print("0\t \t",precision1,"\t",Recall1,"\t",(2*precision1*Recall1/(precision1+Recall1)))
    print("Accuracy = ", ((TP+TN)/(TP+TN+FP+FN)))
    print("Misclassification % = ", (FN+FP)*100/(TP+TN+FP+FN))

In [87]:
#logistic Regression model
from sklearn.linear_model import LogisticRegression
mLR = LogisticRegression(max_iter = 1000)
mLR.fit(xtrain,ytrain)
ypred = mLR.predict(xtest)
test_metrics(ytest,ypred)
validation(ytest,ypred)

accuracy score 0.7586365730078305
confusion matrix
 [[4667  272]
 [1300  274]]
CLS report
               precision    recall  f1-score   support

           0       0.78      0.94      0.86      4939
           1       0.50      0.17      0.26      1574

    accuracy                           0.76      6513
   macro avg       0.64      0.56      0.56      6513
weighted avg       0.71      0.76      0.71      6513

Validation	 precision	 	 Recall 		 F1-score
1	 	 0.5018315018315018 	 0.5018315018315018 	 0.5018315018315018
0	 	 0.7821350762527233 	 0.7821350762527233 	 0.7821350762527233
Accuracy =  0.7586365730078305
Misclassification % =  24.13634269921695


In [88]:
#KNN Classifier n=5
from sklearn.neighbors import KNeighborsClassifier
KNN = KNeighborsClassifier(n_neighbors = 5)
KNN.fit(xtrain,ytrain)
ypred = KNN.predict(xtest)
test_metrics(ytest,ypred)
validation(ytest,ypred)

accuracy score 0.7855059112544143
confusion matrix
 [[4321  618]
 [ 779  795]]
CLS report
               precision    recall  f1-score   support

           0       0.85      0.87      0.86      4939
           1       0.56      0.51      0.53      1574

    accuracy                           0.79      6513
   macro avg       0.70      0.69      0.70      6513
weighted avg       0.78      0.79      0.78      6513

Validation	 precision	 	 Recall 		 F1-score
1	 	 0.5626326963906582 	 0.5626326963906582 	 0.5626326963906582
0	 	 0.8472549019607843 	 0.8472549019607843 	 0.8472549019607843
Accuracy =  0.7855059112544143
Misclassification % =  21.449408874558575


In [89]:
#Desicion tree classifier (gini, depth 7, split = 15)
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(criterion = 'gini', max_depth= 7, min_samples_split=10)
dt.fit(xtrain,ytrain)
ypred = dt.predict(xtest)
test_metrics(ytest,ypred)
validation(ytest,ypred)

accuracy score 0.8240442192538001
confusion matrix
 [[4485  454]
 [ 692  882]]
CLS report
               precision    recall  f1-score   support

           0       0.87      0.91      0.89      4939
           1       0.66      0.56      0.61      1574

    accuracy                           0.82      6513
   macro avg       0.76      0.73      0.75      6513
weighted avg       0.82      0.82      0.82      6513

Validation	 precision	 	 Recall 		 F1-score
1	 	 0.6601796407185628 	 0.6601796407185628 	 0.6601796407185628
0	 	 0.8663318524241839 	 0.8663318524241839 	 0.8663318524241839
Accuracy =  0.8240442192538001
Misclassification % =  17.59557807461999


In [90]:
#random forrest classifier
from sklearn.ensemble import RandomForestClassifier
RF = RandomForestClassifier(n_estimators=100, criterion="entropy", max_depth = 5, min_samples_split = 10)
RF.fit(xtrain,ytrain)
ypred = RF.predict(xtest)
test_metrics(ytest,ypred)
validation(ytest,ypred)

accuracy score 0.8163672654690619
confusion matrix
 [[4602  337]
 [ 859  715]]
CLS report
               precision    recall  f1-score   support

           0       0.84      0.93      0.89      4939
           1       0.68      0.45      0.54      1574

    accuracy                           0.82      6513
   macro avg       0.76      0.69      0.71      6513
weighted avg       0.80      0.82      0.80      6513

Validation	 precision	 	 Recall 		 F1-score
1	 	 0.6796577946768061 	 0.6796577946768061 	 0.6796577946768061
0	 	 0.8427028016846732 	 0.8427028016846732 	 0.8427028016846732
Accuracy =  0.8163672654690619
Misclassification % =  18.36327345309381


In [82]:
#svm Classifier
from sklearn.svm import SVC
SVM = SVC(kernel  = "linear", C=10)
SVM.fit(xtrain,ytrain)
ypred = SVM.predict(xtest)
test_metrics(ytest,ypred)
validation(ytest,ypred)

accuracy score 0.758329494856441
confusion matrix
 [[4939    0]
 [1574    0]]
CLS report
               precision    recall  f1-score   support

           0       0.76      1.00      0.86      4939
           1       0.00      0.00      0.00      1574

    accuracy                           0.76      6513
   macro avg       0.38      0.50      0.43      6513
weighted avg       0.58      0.76      0.65      6513

Validation	 precision	 	 Recall 		 F1-score
1	 	 nan 	 nan 	 nan
0	 	 0.758329494856441 	 0.758329494856441 	 0.758329494856441
Accuracy =  0.758329494856441


D:\coding stuff\Jupyter\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\coding stuff\Jupyter\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\coding stuff\Jupyter\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\9kish\AppData\Local\Temp\ipykernel_10884\20913453

In [92]:
print("SVM misclassification % =", (1574*100/(4939+1574)))

SVM misclassification % = 24.167050514355903


1) The columns are renamed as mentioned in the project statement

2) Null values are not being recognised while using a function checking. However they can be noticed by manually having a glance at the CSV file. And they can be replaced and handled

3) The dataset is split into train and test with test as 20%

4) 
   
   Linear Regression gives accuracy of ------- 75.86365730078305 %

                           misclassification = 24.13634269921695 %
                          
   KNN gives accuracy of --------------------- 78.55059112544143 %
              
                           misclassification = 21.44940887455857 %
                          
   Desicion tree classifier gives accuracy of  82.40442192538001 %
                         
                           misclassification = 17.59557807461999 %
                          
   random forrest classifier gives accuracy of 81.63672654690619 %
                           
                           misclassification = 18.36327345309381 %
                           
   SVM gives accuracy of --------------------- 75.8329494856441  %
                           
                           misclassification = 24.16705051435590 %
   
   Another thing to notice is that SVM classifies all people as below 50k salary people which fails the the validation function as there is division by zero

.



.

5) the income for test data is predicted using the inbuild predict funtion

6) test_metrics is a functioned defined to create confusion matrix and classification report

7) validation is a function defined the validate the values for precision recall and F1

8) misclassification ((FP+FN)/(TP+TN+FP+FN)) is defined in the validation function and produced for all models. All the values are mentioned above

9) Best model of the five is Desicion Tree classifier as it has he highest accuracy of 82.4 %